In [1]:
import pandas as pd
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import Chroma
from langchain.chains import RetrievalQA
from azuremodels import embeddings
df = pd.read_csv(r"C:\Users\GenAIBLRANCUSR35\Documents\scaleBackend\marketConditions.csv")

texts = df.iloc[:, 0].astype(str).tolist()            # first column → content
metadatas = df.iloc[:, 1:].to_dict(orient="records")  # other columns → metadata

PERSIST_DIR = "marketCondtions" 

vectorstore = Chroma.from_texts(
    texts=texts,
    embedding=embeddings,
    metadatas=metadatas,
    collection_name="marketCondtions",
    persist_directory=PERSIST_DIR,
)
PERSIST_DIR = "marketCondtions" 
vectorstore.persist()  
db = vectorstore.as_retriever()
res = db.invoke("""Increased geopolitical risk: escalation between India and Pakistan raises uncertainty in regional markets, likely leading to short-term sell-offs in equities and currency markets across South Asia.

Foreign investor caution: global investors may view the region as riskier, reducing capital inflows into Indian and neighbouring markets, especially in sectors sensitive to stability (e.g. infrastructure, defense, trade-linked industries).

Export disruption & trade costs: heightened border tension may disrupt supply chains and cross-border trade routes, increasing cost of imports/exports, which can affect trade-reliant companies and widen trade deficits.

Insurance/premium & risk-premium rise: companies operating in sensitive sectors may face higher insurance and risk premiums, increasing operational costs and reducing net margins.

Defense and government spending shift: in case of conflict, government budgets may divert toward defense and security spending, potentially crowding out public investment or fiscal support for business and infrastructure — this may dampen growth prospects.

Currency and interest-rate pressure: increased uncertainty or capital outflows may depreciate currency and force tighter monetary policy, raising borrowing costs and impacting debt-heavy firms.

Market volatility & investor sentiment hit: even rumors or escalation can cause heightened volatility — risk-averse investors may move funds to safer assets, reducing liquidity and slowing down stock-market performance.

Longer-term structural risk: repeated conflict cycles can deter long-term foreign direct investment (FDI), slow infrastructure projects, and reduce investor confidence — this can hurt economic growth and corporate valuations.""")

c:\Program Files\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Testing LLM...
LLM Response: Hello! I'm just a virtual assistant, so I don't have feelings, but thank you for asking! How can I assist you today? 😊
✓ LLM working successfully!

Testing Embeddings...
Embedding dimension: 3072
First 5 values: [-0.003638058202341199, 0.0222418662160635, -0.01984875276684761, 0.04138676077127457, 0.006633847486227751]
✓ Embeddings working successfully!


In [10]:
res = db.invoke("Global macroeconomic signals indicate a mixed landscape. Inflation pressures persist in several regions, with central banks maintaining a cautious stance. The Federal Reserve remains watchful, signaling a potential pause in rate adjustments, while the European Central Bank is expected to cut rates further in 2025 amid slowing growth. Liquidity conditions are stabilizing, though vulnerabilities in credit markets remain amid heightened geopolitical risks. Corporate earnings show strength in defense and energy sectors due to escalating conflicts, although broader economic uncertainty weighs on other industries. Geopolitical tensions, including the Russia-Ukraine war, Israel-Iran conflict, and U.S.-China trade frictions, are intensifying market volatility, driving a risk-off sentiment with capital flowing into safe-haven assets like gold and government bonds. Crude oil prices remain under pressure from OPEC+ production increases and slowing global demand but are vulnerable to further geopolitical disruptions. Regulatory actions are increasing globally to mitigate financial system risks, with a focus on liquidity management and asset price stability. Recession risks persist, particularly in the eurozone, where growth forecasts have been downgraded. In the U.S., a resilient labor market and consumer spending provide some support, though uncertainty lingers. Investor sentiment is cautious, with valuations pressured by geopolitical concerns and rising interest rates, underscoring a precarious macroeconomic environment.")

In [11]:
res[0].metadata

{'Silver': 0,
 'EQ_Pharma': 0.375,
 'MF_Flexi': 0.67,
 'EQ_MetalsMining': 0.375,
 'MF_Index': 0.67,
 'EQ_Defense': 0.375,
 'EQ_OilGas': 0.375,
 'BankFD': -2.5,
 'EQ_IT': 0.375,
 'MF_SmallCap': 0.375,
 'EQ_Banking': 0.375,
 'Gold': 12,
 'EQ_FMCG': 0.375,
 'Crypto': -8,
 'RealEstate': 2,
 'DebtBond': 0.67,
 'EQ_Automobile': 0.375}

In [3]:
import pandas as pd

def apply_instrument_deltas(strategy_csv, output_csv, deltas: dict):
    """
    Applies instrument-level deltas directly and normalizes totals back to 100%.
    
    Assumptions:
    - strategy.csv contains all instrument columns as numeric weights summing to 100.
    - 'Client Type' remains untouched.
    - deltas dict keys must match exact CSV column names.
    """
    df = pd.read_csv(strategy_csv)

    # instrument columns = all except Client Type
    instrument_cols = [c for c in df.columns if c != "Client Type"]

    # Validate column presence
    missing = [k for k in deltas if k not in instrument_cols]
    if missing:
        raise ValueError(f"Delta keys not found in strategy columns: {missing}")

    # ---- Apply deltas ----
    for inst_col in instrument_cols:
        if inst_col in deltas:
            base = df[inst_col].astype(float)
            pct = deltas[inst_col]
            df[f"{inst_col}_adj"] = base + (base * pct/100.0)
        else:
            df[f"{inst_col}_adj"] = df[inst_col].astype(float)

    adj_cols = [f"{c}_adj" for c in instrument_cols]

    # ---- Normalize so total remains 100 ----
    df["row_total"] = df[adj_cols].sum(axis=1)
    df["row_total"] = df["row_total"].replace(0, 1.0)

    for col in instrument_cols:
        df[f"{col}_final"] = df[f"{col}_adj"] / df["row_total"] * 100.0

    # ---- Keep CLEAN output ----
    final_cols = ["Client Type"] + [f"{c}_final" for c in instrument_cols]

    df[final_cols].to_csv(output_csv, index=False)
    return df[final_cols]


In [4]:
if __name__ == "__main__":
    deltas = res[0].metadata

    updated = apply_instrument_deltas(
        "generalStrategyStandard.csv",
        "strategy_updated.csv",
        deltas
    )

    print(updated)


       Client Type  FixedDeposits_Total_final  BankFD_final  DebtBond_final  \
0     Conservative                  25.588900     16.206303        8.870819   
1         Balanced                  14.632633      7.943429        6.521973   
2  Growth-Oriented                   5.343647      2.538232        2.778696   
3    Strategic/HNI                   2.953850      1.403079        1.536002   

   MutualFunds_Total_final  MF_Index_final  MF_Flexi_final  MF_SmallCap_final  \
0                11.372844        5.913879        4.731103           1.158609   
1                16.723009        6.521973        5.978476           4.791534   
2                18.702763        6.668871        6.668871           5.988224   
3                14.769249        5.529607        4.915206           4.814775   

   Equity_Total_final  EQ_Banking_final  ...  EQ_FMCG_final  \
0            4.549138          1.158609  ...       0.579304   
1           10.451881          1.597178  ...       1.863374   
2        